In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, classification_report, RocCurveDisplay, roc_curve, auc

from confidence_intervals import evaluate_with_conf_int
from confidence_intervals.utils import barplot_with_ci

import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

In [2]:
prediction_df = pd.read_csv('ModelPredictions.csv', index_col=0)
prediction_df.head()

,file,ground truth,LR_FeatureStates_Age_Sex.pkl,LR_FeatureStates_Age_Sex_Symptoms.pkl,LR_FeatureStates_Rec_Only.pkl,LR_FeatureStates_Symptoms.pkl,LR_MFCC_Age_Sex.pkl,LR_MFCC_Age_Sex_Symptoms.pkl,LR_MFCC_Rec_Only.pkl,LR_MFCC_Symptoms.pkl,...,SVM_FeatureStates_Symptoms_Age_Sex.pkl,SVM_MFCC_Age_Sex.pkl,SVM_MFCC_Age_Sex_Symptoms.pkl,SVM_MFCC_Rec_Only.pkl,SVM_MFCC_Symptoms.pkl,SVM_OpenSmile_Age_Sex.pkl,SVM_OpenSmile_Age_Sex_Symptoms.pkl,SVM_OpenSmile_Rec_Only.pkl,SVM_OpenSmile_Symptoms.pkl,Sex
0,Atrophy-0001apo,Benign,Benign,Benign,Benign,Benign,Benign,Benign,Benign,Benign,...,Benign,Benign,Benign,Benign,Benign,Benign,Benign,Benign,Benign,2
1,Atrophy-0400og6,Benign,Benign,Benign,Benign,Benign,Benign,Malignant,Benign,Malignant,...,Benign,Benign,Malignant,Benign,Malignant,Benign,Benign,Benign,Benign,2
2,Atrophy-0500cg6,Benign,Benign,Malignant,Benign,Malignant,Malignant,Malignant,Benign,Malignant,...,Malignant,Benign,Malignant,Benign,Malignant,Malignant,Malignant,Malignant,Malignant,1
3,Atrophy-0601ntf,Benign,Malignant,Malignant,Malignant,Malignant,Malignant,Malignant,Malignant,Malignant,...,Malignant,Malignant,Malignant,Malignant,Benign,Benign,Benign,Benign,Benign,1
4,Atrophy-07017ip,Benign,Benign,Benign,Benign,Benign,Benign,Benign,Benign,Benign,...,Benign,Benign,Benign,Benign,Benign,Benign,Malignant,Benign,Malignant,2


In [3]:
def claculate_sensitivity(y_test, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=['Benign', 'Malignant']).ravel()
    sensitivity = tp/(tp+fn)
    return sensitivity

In [4]:
def claculate_specificity(y_test, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred, labels=['Benign', 'Malignant']).ravel()
    specificity = tn / (tn+fp)
    return specificity

In [ ]:
barWidth=0.25

cmap = plt.get_cmap('Pastel1')
colors = ['tab:Blue', 'tab:Orange']

for model in models:
    fig, ax = plt.subplots()
    j=0
    x=0
    
    feature_ticks = []
    feature_tick_labels = []
    input_ticks = []
    input_tick_labels = []
    
    data = df.loc[(model, slice(None), 'Balanced Accuracy')]
    
    for input_val in input_vals:
        input_ticks.append(x+0.2)
        input_tick_labels.append(input_val.replace(' + ', '\n+ '))
        x+=0.05
        for feature in features:
            feature_ticks.append(x+0.2)
            feature_tick_labels.append(feature)
            x+=0.1
            for dataset in datasets:
                i=j%len(datasets)
                values = data.loc[input_val][dataset][feature]
                mean_val = float(values[0])
                lower_ci = float(values[1][0])
                upper_ci = float(values[1][1])
    
                ax.bar(x, mean_val, width=barWidth, color=colors[i], label=dataset)
                ax.vlines(x=x, ymin=lower_ci, ymax=upper_ci, colors='k')
                j+=1
                x+=0.25
    
    plt.xticks(feature_ticks, feature_tick_labels, rotation=90)
    for n in range(len(input_ticks)):
        plt.text(input_ticks[n], -0.4, input_tick_labels[n], fontsize=16, ha='left', ma='center', va='top')

    if model == 'NN':
        model = 'MLP'
    plt.title(f'{model} Balanced Accuracy')
    plt.ylabel('Balanced Accuracy')
    legend_without_duplicate_labels(ax, loc='lower left')
    plt.savefig(f'{model}_compare_datasets.pdf')
    plt.show()